In [ ]:
import os
import sys
sys.path.append('../code/')

from nn_models import *
from nn_train_utils import *
from kinematics_decoding import load_model

from collections import defaultdict
import glob
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

In [ ]:
conv_folders = glob.glob('./experiment_4/*_*_*')

conv_keylist = ['arch_type', 'nlayers', 'n_skernels', 'n_tkernels', 
                's_kernelsize', 't_kernelsize', 's_stride', 't_stride', 'is_trained', 'seed',
                'validation_accuracy']

In [ ]:
conv_dict = defaultdict(list)

for folder in conv_folders:
    try:
        with open(os.path.join(folder, 'config.yaml')) as myfile:
            myconfig = yaml.load(myfile)    
    except:
        print(folder)    
    for key in conv_keylist:
        try:
            conv_dict[key].append(myconfig[key])
        except:
            if key == 'is_trained':
                if folder[-1] == 'r':
                    conv_dict[key].append(False)
                else:
                    conv_dict[key].append(True)

In [ ]:
all_conv_models = pd.DataFrame(conv_dict)

all_conv_models = all_conv_models.astype({'nlayers': int, 
                                          's_kernelsize':int, 
                                          't_kernelsize': int, 
                                          's_stride': int, 
                                          't_stride': int})

In [ ]:
# Load train dataset as test
PATH_TO_DATA = os.path.abspath('../dataset/')
train_dataset = Dataset(os.path.join(PATH_TO_DATA, 'pcr_dataset_train.hdf5'), dataset_type='test')

conv_train_accuracies = [evaluate_model(
    load_model(all_conv_models.iloc[i], 4, 'conv', all_conv_models['is_trained'].iloc[i]),
    train_dataset) for i in range(all_conv_models.shape[0])]

In [ ]:
# Load test dataset
PATH_TO_DATA = os.path.abspath('../dataset/')
test_dataset = Dataset(os.path.join(PATH_TO_DATA, 'pcr_dataset_test.hdf5'), dataset_type='test')

conv_test_accuracies = [evaluate_model(
    load_model(all_conv_models.iloc[i], 4, 'conv', all_conv_models['is_trained'].iloc[i]),
    test_dataset) for i in range(all_conv_models.shape[0])]

In [ ]:
conv_val_accuracies = all_conv_models['validation_accuracy'].tolist()
all_conv_models['test_accuracy'] = pd.Series(conv_test_accuracies)
all_conv_models['train_accuracy'] = pd.Series(conv_train_accuracies)

In [ ]:
all_conv_models.to_pickle('analyzed_convmodels.p')